In [ ]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from math import ceil

BS = 128
IMAGE_DIMS = (96, 96, 3)

def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

# construct the argument parse and parse the arguments
if not is_interactive():
    ap = argparse.ArgumentParser()
    ap.add_argument("-d", "--dataset", required=True,
        help="path to train.csv")
    ap.add_argument("-m", "--model", required=True,
        help="path to output model")
    ap.add_argument("-l", "--labelbin", required=True,
        help="path to output label binarizer")
    args = vars(ap.parse_args())
else:
    args = {}
    args['dataset'] = '../train.csv'
    args['model'] = 'mobile_categorizer.model'
    args['labelbin'] = 'lb.pickle'

model_path = 'weights-improvement-02-0.96.model'
dataset = pd.read_csv(args['dataset'])

dataset = dataset[dataset
                      .image_path
                      .apply(lambda image_path: image_path.startswith('mobile_image'))]


dataset.Category = dataset.Category.apply(str)

unique_categories = dataset['Category'].unique().tolist()


print("[INFO] loading network...")
model = load_model(model_path)

# construct the image generator for data augmentation
image_data_gen = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest", rescale=1/255.0, dtype=float)


dataset_iter = image_data_gen.flow_from_dataframe(dataset,
                                                    directory='./',
                                                    x_col='image_path',
                                                    y_col='Category',
                                                    target_size=(IMAGE_DIMS[1], IMAGE_DIMS[0]),
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    classes=unique_categories,
                                                    batch_size=BS)

classifications = model.predict_generator(dataset_iter, 
                                          steps=ceil(len(dataset) / BS),
                                          use_multiprocessing=True)
display(classifications)



[INFO] loading network...
Found 160330 images belonging to 27 classes.
